In [ ]:
#Conexão Banco de Dados
import cx_Oracle
import retorna_latlong as geolocalizacao
from dotenv import dotenv_values

config = dotenv_values() #Variaves de configuração

localizacao = geolocalizacao.lat_long(API_KEY = config['API_KEY'],
                                      DOMAIN = 'maps.googleapis.com')

connection = cx_Oracle.connect(user=config['USER'], 
                               password=config['PASSWORD'], 
                               dsn=config['DSN'], 
                               encoding="UTF-8")

In [ ]:
cursor = connection.cursor()
cur = connection.cursor()
result = []

cursor.execute ("SELECT * FROM ESTV_CLIENTE_LAT_LONG ORDER BY seqPessoa ASC")


while True:
    row = cursor.fetchone()
    
    if row is None:
        break
    
    cur.execute("INSERT INTO EST_CLIENTE_LAT_LONG VALUES (:SEQPESSOA, :LATITUDE, :LONGITUDE)",
                [row[0], localizacao.ret_lat_ou_long(row[7],"lat"),localizacao.ret_lat_ou_long(row[7],"long") ])
    connection.commit()


In [ ]:
query = """
                    BEGIN
                    FOR I IN(select * FROM EST_CLIENTE_LAT_LONG WHERE IMPORTADO = 0)
                        LOOP
                        UPDATE GE_PESSOA SET LATITUDE = I.LATITUDE, LONGITUDE = I.LONGITUDE
                        WHERE SEQPESSOA = I.SEQPESSOA;
                        
                        INSERT INTO DGE_OCORRENCIA (TIPO,NIVEL,DATA,HORA,CODUSUARIO,DETALHE,MOTIVO,SISTEMA,MODULO,TABLINK,TERMINAL,
                                USUARIOOS,CODAPLICACAO,SESSIONID,CODLINK)
                                VALUES(2,1,TRUNC(SYSDATE),172600,'RODOLFO.BORTOLOZO','LATITUDE E LONGITUDE',
                                'LATITUDE E LONGITUDE','GLOBAL','PESSOA','GE_PESSOA','DESKTOP-1U8AVLM','rodolfo.bortolozo','GEPES0010',NULL, I.SEQPESSOA);
                        
                        UPDATE EST_CLIENTE_LAT_LONG SET IMPORTADO = 1 WHERE SEQPESSOA = I.SEQPESSOA;
                        
                        COMMIT;
                        
                        END LOOP;
                    END;  
"""

cursor.execute(query)

connection.commit()